In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

try:
  import google.colab
  IN_COLAB = True
  from google.colab.data_table import DataTable
  DataTable.max_columns = 102
except:
  IN_COLAB = False

# Data Cleaning

In [ ]:
df = pd.read_csv("Harbor_Water_Quality.csv", low_memory=False)

In [ ]:
df.head()

In [ ]:
# Drop duplicate samples
df["Duplicate Sample"] = df["Duplicate Sample"].replace(np.NaN, False)
df["Duplicate Sample"] = df["Duplicate Sample"].replace('D', True)
df = df.drop(list(df[(df["Duplicate Sample"] == True)].index))

In [ ]:
# Collect basic water quality metrics
df = pd.DataFrame({
    "Sampling Location": df["Sampling Location"],
    "Sample Date": df["Sample Date"],
    "Season": np.NaN, # Integers Winter = 1, Spring = 2 ... 
    "Top Sample Temperature": df["Top Sample Temperature"],
    "Bottom Sample Temperature": df["Bottom Sample Temperature"],
    "Top Salinity": df["Top Salinity(psu)"],
    "Bottom Salinity": df["Bottom Salinity(psu)"],
    "Top PH": df["Top PH"],
    "Bottom PH": df["Bottom PH"],
    "Long": df["Long"],
    "Lat": df["Lat"],
})

In [ ]:
df.head()

In [ ]:
def filter_floats(value):
  # Some numeric data points are corrupted, lets remove them.
  try:
    return float(value)
  except ValueError:
    return np.NaN

In [ ]:
df["Top PH"] = df["Top PH"].apply(lambda value: filter_floats(value))
df["Bottom PH"] = df["Bottom PH"].apply(lambda value: filter_floats(value))
df["Top Sample Temperature"] = df["Top Sample Temperature"].apply(lambda value: filter_floats(value))
df["Bottom Sample Temperature"] = df["Bottom Sample Temperature"].apply(lambda value: filter_floats(value))
df["Top Salinity"] = df["Top Salinity"].apply(lambda value: filter_floats(value))
df["Bottom Salinity"] = df["Bottom Salinity"].apply(lambda value: filter_floats(value))
df["Long"] = df["Long"].apply(lambda value: filter_floats(value))
df["Lat"] = df["Lat"].apply(lambda value: filter_floats(value))

In [ ]:
df["Sample Date"] = df["Sample Date"].apply(lambda date: pd.to_datetime(date))

# Adding Features

In [ ]:
def season(date):
    if date.month == 12 or 1 <= date.month <= 2:
        return 1 # "Winter"
    if 3 <= date.month <= 5:
        return 2 # "Spring"
    if 6 <= date.month <= 8:
        return 3 # "Summer"
    if 9 <= date.month < 12:
        return 4 # "Fall"

In [ ]:
df["Season"] = [season(date) for date in df["Sample Date"]]

In [ ]:
df = df.sort_values("Sample Date")

In [ ]:
df.head()

# Exploration & Sampling

In [ ]:
wq_data = df[["Sample Date", "Bottom PH", "Bottom Sample Temperature", "Bottom Salinity"]]

In [ ]:
univariate_ph = wq_data[["Sample Date", "Bottom PH"]].dropna().sort_values(by="Sample Date")
univariate_ph.head()

In [ ]:
univariate_ph.info()

In [ ]:
ten_year_avg = wq_data[["Sample Date", "Bottom PH"]].dropna().sort_values(by="Sample Date")
ten_year_avg[(ten_year_avg["Sample Date"] > "2012-1-1")]["Bottom PH"].mean()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=univariate_ph, x="Sample Date", y="Bottom PH", s=5)
plt.ylim(6, 9)
plt.ylabel("pH")

In [ ]:
seasons = df[["Sample Date", "Season", "Bottom PH"]].dropna().sort_values(by="Sample Date")
seasons["Sample Date"] = seasons["Sample Date"].apply(lambda date: pd.to_datetime(date))
seasons["Sample Date"] = seasons["Sample Date"].apply(lambda date: date.year)

In [ ]:
winter = seasons[(seasons["Season"] == 1)]
spring = seasons[(seasons["Season"] == 2)]
summer = seasons[(seasons["Season"] == 3)]
fall = seasons[(seasons["Season"] == 4)]

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=winter, x="Sample Date", y="Bottom PH", color='skyblue')
plt.title("Winter")
plt.ylim(6, 9)
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=spring, x="Sample Date", y="Bottom PH", color='lightgreen')
plt.title("Spring")
plt.ylim(6, 9)
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=summer, x="Sample Date", y="Bottom PH", color='gold')
plt.title("Summer")
plt.ylim(6, 9)
plt.xticks(rotation=90);

In [ ]:
reproduction_seasons = pd.concat([spring, summer])
reproduction_seasons[(reproduction_seasons["Sample Date"] > 2010)]["Bottom PH"].mean()

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=fall, x="Sample Date", y="Bottom PH", color='orange')
plt.title("Fall")
plt.ylim(6, 9)
plt.xticks(rotation=90);

# Time Series Forecasting with Prophet

In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(data=univariate_ph, x="Sample Date", y="Bottom PH", s=10)
plt.ylim(6, 9)
plt.ylabel("pH")

In [ ]:
univariate_ph = univariate_ph.rename(
    columns={
        "Sample Date": "ds",
        "Bottom PH": "y"
    }
)

In [ ]:
# Group all sample dates to their median pH
univariate_ph = univariate_ph.groupby(['ds'], as_index=False).median()

In [ ]:
univariate_ph

In [ ]:
sns.scatterplot(data=univariate_ph, x="ds", y="y", s=5)
plt.ylim(6, 9)
plt.xlabel("Date")
plt.ylabel("pH")

In [ ]:
split = '2021-12-13'

In [ ]:
ph_train = univariate_ph[(univariate_ph["ds"] <= split)]
ph_test = univariate_ph[(univariate_ph["ds"] >= split)]

In [ ]:
model = Prophet()

In [ ]:
model.fit(ph_train)

In [ ]:
forecast_horizon = 3650

In [ ]:
future = model.make_future_dataframe(periods=forecast_horizon)
future.tail()

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
fig = model.plot(forecast, ax=ax)
plt.ylim(6, 9)
plt.xlabel("Year")
plt.ylabel("pH")
ax.set_title('10 Year Time Series Forecast using Prophet')
plt.show()

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(model, forecast)

In [ ]:
fig2 = model.plot_components(forecast)

## Evaluation

Information on how evaluation works: https://facebook.github.io/prophet/docs/diagnostics.html

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(data=univariate_ph, x="ds", y="y", s=10)
sns.scatterplot(data=forecast, x="ds", y="yhat", palette="deep", alpha=0.5, s=10)

real_patch = mpatches.Patch(color='blue', label='Actual Values')
pred_patch = mpatches.Patch(color='orange', label='Predicted Values')
plt.legend(handles=[real_patch, pred_patch])

plt.ylim(6.5, 8.75)
plt.xlabel("Date")
plt.ylabel("pH")
plt.title("10 Year Forecast Vs. Actual Samples of New York Harbor", fontweight="bold")

In [ ]:
univariate_ph_focus = univariate_ph[(univariate_ph["ds"] >= '2017-1-1') & (univariate_ph["ds"] <= '2018-1-1')]
forecast_focus = forecast[(forecast["ds"] >= '2017-1-1') & (forecast["ds"] <= '2018-1-1')]
sns.scatterplot(data=univariate_ph_focus, x="ds", y="y", s=15)
sns.scatterplot(data=forecast_focus, x="ds", y="yhat", palette="deep", alpha=0.75, s=15)
plt.ylim(6, 9)
plt.xlabel("Date")
plt.xticks(rotation=90)
plt.ylabel("pH")
plt.title('2017-2028 Forecast vs. Actual Values')

In [ ]:
from prophet.diagnostics import performance_metrics

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='730 days', period='180 days', horizon='3650 days')
# df_cv = cross_validation(model, initial='730 days', period='120 days', horizon='120 days')

In [ ]:
df_cv

In [ ]:
df_cv.to_csv("error-metrics.csv")

In [ ]:
df_p = performance_metrics(df_cv)

In [ ]:
df_p = performance_metrics(df_cv)
df_p

In [ ]:
df_p[(df_p["horizon"] < "1826 days")]

Relevant metrics for forecasting models are:

  **MSE**: The mean squared error (MSE) of an estimator (of a procedure for estimating an unobserved quantity) measures the average of the squares of the errors — that is, the average squared difference between the estimated values and what is estimated. MSE is a risk function, corresponding to the expected value of the squared error loss. The fact that MSE is almost always strictly positive (and not zero) is because of randomness or because the estimator does not account for information that could produce a more accurate estimate.

  **RMSE**: The root-mean-squared error is the square root of the average squared difference between the target and predicted values. RMSE is more sensitive to outliers than MAE,so if you're concerned about large errors, then RMSE can be a more useful metric to evaluate. Similar to MAE, a smaller value indicates a higher quality model (0 represents a perfect predictor).

  **MAE**: The mean absolute error (MAE) is the average absolute difference between the target values and the predicted values. This metric ranges from zero to infinity; a lower value indicates a higher quality model.

  **MAPE**: Mean absolute percentage error (MAPE) is the average absolute percentage difference between the labels and the predicted values. This metric ranges between zero and infinity; a lower value indicates a higher quality model.
  MAPE is not shown if the target column contains any 0 values. In this case, MAPE is undefined.
  
### Cross Validation

In our cross validation model we initally start our training with 2 years of training data, set our cutoff point and forecast recursivly for the next 10 years (until 10 year forecasts can NOT be made from a set cutoff point because there is no data to compare to). Per forecast cycle we then set our cutoff point to the next 6 months from the prior cutoff point (180 days) and continue the cycle.

# GeoSpatial Visualization

This dataset contains all measurements with coordinate values. This dataset ranges from 1986-2021

A few years contain missing data:

2000-2004

2014-2015


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
geo_default = df[["Sample Date", "Bottom PH", "Long", "Lat"]].dropna()

Some Longditude and Latitude Values are swapped.

In [ ]:
idx = (df['Lat'] < 40)

# This should catch the majority of cases of where the lat and long are swapped.
geo_default.loc[idx,['Long','Lat']] = geo_default.loc[idx,['Lat','Long']].values
geo_default

## Visual

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def f(year):
    import warnings
    warnings.filterwarnings('ignore')
    
    geo_year_select = geo_default[(geo_default["Sample Date"] >= f'{year}-1-1') & (geo_default["Sample Date"] <= f'{year+1}-1-1')]
    # Random coordinate variability for Sites to see all samples on map
    variability = 0.005
    geo_year_select["Long"] = geo_year_select["Long"].apply(lambda long: long + np.random.uniform(0, variability));
    geo_year_select["Lat"] = geo_year_select["Lat"].apply(lambda long: long + np.random.uniform(0, variability));
    
    fig = px.scatter_mapbox(geo_year_select, 
                            lat="Lat", 
                            lon="Long",
                            color="Bottom PH",
                            range_color=[5.5,9],
                            color_continuous_scale=px.colors.sequential.Hot,
                            zoom=9, 
                            center={"lat": 40.6928, "lon": -74.0120}
                            )
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(
        title_x=0.6,
        title_y=0.05,
    )
    fig.show()
    #return geo_year_select

interact(f, year=widgets.IntSlider(min=2005, max=2021, step=1, value=2005));